In [91]:
import boto3
import os

In [92]:
#variables
OHIO_REGION = "us-east-2"
NORTH_VIRGINIA = "us-east-1"
AMI_UBUNTU_OHIO = "ami-020db2c14939a8efb"
AMI_UBUNTU_NORTH_VIRGINIA = "ami-0279c3b3186e54acd"
KEY_PAIR_OHIO_NAME = "key_pair_marcelo_ohio"
KEY_PAIR_NORTH_VIRGINIA_NAME = "key_pair_marcelo_north_virginia"
SEC_GROUP_NAME_DB = "Sec_Group_DB"
SEC_GROUP_NAME_DJ = "Sec_Group_DJ"

In [93]:
# commands line
comand_db="""
#cloud-config

runcmd:
- cd /
- sudo apt update
- sudo apt install postgresql postgresql-contrib -y
- sudo su - postgres
- sudo -u postgres psql -c "CREATE USER cloud WITH PASSWORD 'cloud';"
- sudo -u postgres psql -c "CREATE DATABASE tasks;"
- sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE tasks TO cloud;"
- sudo echo "listen_addresses = '*'" >> /etc/postgresql/10/main/postgresql.conf
- sudo echo "host all all 0.0.0.0/0 trust" >> /etc/postgresql/10/main/pg_hba.conf
- sudo ufw allow 5432/tcp -y
- sudo systemctl restart postgresql

"""

In [94]:
# permissions
IpPermissions_DB = [
    {
        'IpProtocol': 'tcp',
        'FromPort': 22,
        'ToPort': 22,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 5432,
        'ToPort': 5432,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    },
    {
            "FromPort": 80,
            "ToPort": 80,
            "IpProtocol": "tcp",
            "IpRanges": [
                {"CidrIp": "0.0.0.0/0", "Description": "internet"},
            ],
        },
]


IpPermissions_DJ = [
    {
        'IpProtocol': 'tcp',
        'FromPort': 22,
        'ToPort': 22,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 8080,
        'ToPort': 8080,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    },
    {
            "FromPort": 80,
            "ToPort": 80,
            "IpProtocol": "tcp",
            "IpRanges": [
                {"CidrIp": "0.0.0.0/0", "Description": "internet"},
            ],
        },
]


In [95]:
#create a key_pair
def create_key_pair(client,key_name:str):
    key_pair = client.create_key_pair(KeyName=key_name)

    private_key = key_pair["KeyMaterial"]

    # write private key to file with 400 permissions
    print(f"Writing {key_name}.pem in directory ./")
    with os.fdopen(os.open(f"./{key_name}.pem", os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle:
        handle.write(private_key)
    return key_pair

#delete a key_pair
def delete_key_pair(client,key_name:str):
    key_pair = client.delete_key_pair(KeyName=key_name)
    print(f"removing {key_name} if exist")
    # write private key to file with 400 permissions
    try:
        os.remove("./" + key_name + ".pem")
        
    except:
        pass


def create_an_instance(client, ami,key_name,user_data,sec_group_name,sec_group_id):
    
    instancetype = "t2.micro"
    print(f"Creating an instance with {instancetype}, and keyname {key_name['KeyName']}")
    if user_data == None:
        instances = client.run_instances(
            ImageId=ami,
            MinCount=1,
            MaxCount=1,
            InstanceType="t2.micro",
            KeyName=key_name['KeyName'],
            SecurityGroupIds=[sec_group_id],
            SecurityGroups=[sec_group_name]
        )
    else:
        
        instances = client.run_instances(
            ImageId=ami,
            MinCount=1,
            MaxCount=1,
            InstanceType="t2.micro",
            KeyName=key_name['KeyName'],
            UserData = user_data,
            SecurityGroupIds=[sec_group_id],
            SecurityGroups=[sec_group_name]
        )
    for i in instances['Instances']:
        if i['KeyName'] == key_name["KeyName"]:
            instance_id = i['InstanceId']
    return instances, instance_id

def delete_instances(client,key_pair):
    list_instances_id = []
    for i in client.describe_instances(Filters=[
        {
            'Name': 'key-name',
            'Values': [
                key_pair,
            ]
        },
        {
            'Name': 'instance-state-name',
            'Values': [
                "pending","running","stopping","stopped"
            ]
        },
    ],
)['Reservations']:
        list_instances_id.append(i['Instances'][0]["InstanceId"])
    if len(list_instances_id) > 0:
        print(f"Deleting Instances ids {list_instances_id}")
        waiter = client.get_waiter('instance_terminated')
        client.terminate_instances(InstanceIds=list_instances_id)
        waiter.wait(InstanceIds=list_instances_id)
    else:
        print(f"There is no instances not terminated with the keypair {key_pair}")
        
def delete_sec_group(client,sec_group_name):
    find_sec_group = False
    security_groups = client.describe_security_groups()
    for sec_group in security_groups['SecurityGroups']:
        if sec_group['GroupName'] == sec_group_name:
            find_sec_group = True
            
            deleted_sec_group = client.delete_security_group(GroupName=sec_group["GroupName"], GroupId=sec_group["GroupId"])
    if find_sec_group:
        print(f"{sec_group_name} deleted")
    else:
        print(f"{sec_group_name} not found")
def create_security_group(client, SEC_GROUP_NAME,IP_PERMISSIONS):
    vpcs = client.describe_vpcs()
    vpc_id = vpcs['Vpcs'][0]['VpcId']
    security_group = client.create_security_group(GroupName=SEC_GROUP_NAME,
                                                Description="Created with boto3",
                                                VpcId=vpc_id)
    security_group_id = security_group['GroupId']
    client.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions = IP_PERMISSIONS
    )
    print(f"Security Group {SEC_GROUP_NAME} created")
    return security_group_id

In [96]:

#delete_sec_group(ec2_ohio,SEC_GROUP_DB)

In [97]:
#checking if boto3 client is working
iam = boto3.client("iam")

for user in iam.list_users()["Users"]:
    if user["UserName"] == "marcelo":
        print(user["UserName"])
        print(user["UserId"])
        print(user["Arn"])
        print(user["CreateDate"])

marcelo
AIDATTPQFU3W27CNBJLEK
arn:aws:iam::248001177325:user/marcelo
2021-09-21 18:22:34+00:00


In [98]:
#connection to ec2
ec2_ohio = boto3.client('ec2', region_name='us-east-2')
ec2_north_virginia_ = boto3.client('ec2', region_name='us-east-1')

In [99]:
delete_instances(ec2_ohio,KEY_PAIR_OHIO_NAME)
delete_instances(ec2_north_virginia_,KEY_PAIR_NORTH_VIRGINIA_NAME)

Deleting Instances ids ['i-005928d3b186d7c27']
There is no instances not terminated with the keypair key_pair_marcelo_north_virginia


In [100]:
delete_key_pair(ec2_ohio,KEY_PAIR_OHIO_NAME)
delete_key_pair(ec2_north_virginia_,KEY_PAIR_NORTH_VIRGINIA_NAME)

removing key_pair_marcelo_ohio if exist
removing key_pair_marcelo_north_virginia if exist


In [101]:
delete_sec_group(ec2_ohio,SEC_GROUP_NAME_DB)
delete_sec_group(ec2_north_virginia_,SEC_GROUP_NAME_DJ)

Sec_Group_DB deleted
Sec_Group_DJ deleted


In [102]:
sec_group_id_db = create_security_group(ec2_ohio,SEC_GROUP_NAME_DB,IpPermissions_DB)
sec_group_id_dj = create_security_group(ec2_north_virginia_,SEC_GROUP_NAME_DJ,IpPermissions_DJ)

Security Group Sec_Group_DB created
Security Group Sec_Group_DJ created


In [103]:
key_pair_ohio = create_key_pair(ec2_ohio,KEY_PAIR_OHIO_NAME)
key_pair_north_virginia = create_key_pair(ec2_north_virginia_,KEY_PAIR_NORTH_VIRGINIA_NAME)

Writing key_pair_marcelo_ohio.pem in directory ./
Writing key_pair_marcelo_north_virginia.pem in directory ./


In [104]:
instances_ohio, instance_ohio_id = create_an_instance(ec2_ohio,AMI_UBUNTU_OHIO,key_pair_ohio,comand_db,SEC_GROUP_NAME_DB,sec_group_id_db)
#instances_north_virginia, instance_north_virginia_id = create_an_instance(ec2_north_virginia_,AMI_UBUNTU_NORTH_VIRGINIA,key_pair_north_virginia)

Creating an instance with t2.micro, and keyname key_pair_marcelo_ohio
